In [1]:
import torch
from torch import nn as nn

In [44]:
class Transformer(nn.Module):
    def __init__(self, 
                src_vocab_size,
                trg_vocab_size,
                src_pad_idx, 
                trg_pad_idx, 
                embed_size = 512,
                heads = 8,
                num_layers = 6,
                forward_expansion = 0,
                dropout = 0,
                device = 'cpu',
                max_length = 50, ):
        
        super(Transformer, self).__init__()
        
        self.encoder = Encoder(src_vocab_size,
                              embed_size,
                              num_layers,
                              device,
                              heads,
                              dropout,
                              forward_expansion,
                              max_length, )
        
        self.decoder = Decoder(src_vocab_size,
                              embed_size,
                              num_layers,
                              device,
                              heads,
                              dropout,
                              forward_expansion,
                              max_length, )
        
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)
        
    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril((torch.ones(trg_len, trg_len))).expand(N, 1, trg_len, trg_len)
        # lower triagle of the N-dim tensor
        return trg_mask.to(self.device)
            
    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)                      
        out = self.decoder(trg, enc_src, src_mask, trg_mask)                      
        return out                          

In [49]:
class EncoderBlock(nn.Module):
    
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(EncoderBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        
        self.feed_forward = nn.Sequential(nn.Linear(embed_size, forward_expansion * embed_size),
                                          nn.ReLU(),
                                          nn.Linear(forward_expansion * embed_size, embed_size))       
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, query, key, value, mask):
        attention = self.attention(query, key, value, mask)
        
        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out
    
    
class Encoder(nn.Module):
    
    def __init__(self,
                src_vocab_size,
                embed_size,
                num_layers,
                device,
                heads,
                forward_expansion,
                dropout,
                max_length, ):
        
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self_device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.positional_embedding = nn.Embedding(max_length, embed_size)
        
        self.layers = nn.ModuleList(
                            [
                                EncoderBlock(embed_size,
                                            heads, 
                                            dropout = dropout,
                                            forward_expansion = forward_expansion,)
                               
                                for _ in range(num_layers)
                            ])
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        N, seq_len = x.shape
        positions = torch.arange(0, seq_len).expand(N, seq_len).to(device)
        out = self.dropout(
              (self.word_embedding(x) + self.positional_embedding(positions)))
            
        # In the Encoder the query, key, value are all the same, it's in the decoder this will change. 
        for layer in self.layers:
            out = layer(out, out, out, mask)
       
        return out

In [61]:
class DecoderBlock(nn.Module):
    
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads = heads)
        self.encoder_block = EncoderBlock(embed_size, heads, dropout, forward_expansion)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.encoder_block(query, key, value, src_mask)
        
        return out
    
class Decoder(nn.Module):
    
    def __init__(self,
                src_vocab_size,
                embed_size,
                num_layers,
                device,
                heads,
                forward_expansion,
                dropout,
                max_length, ):
        
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.positional_embedding = nn.Embedding(max_length, embed_size)
        
        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ] )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_len = x.shape
        positions = torch.arange(0, seq_len).expand(N, seq_len).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.positional_embedding(positions)))
            
        for layer in self.layers:
             x = layer(x, enc_out, enc_out, src_mask, trg_mask)
                
        out = self.fc_out(x)
            
        return out

In [57]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.heads = heads
        self.embed_size = embed_size
        self.head_dim = embed_size // heads
        
        assert (
        self.head_dim * heads == embed_size), "Embedding size needs to be divisible by heads"
        
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
        
    def forward(self, query, keys, values, mask):
        # Get number of training examples
        N = query.shape[0]
                
        # Split the embedding into self.heads different pieces
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)
        
        values = self.values(values)  # (N, value_len, heads, head_dim)
        keys = self.keys(keys)  # (N, key_len, heads, head_dim)
        queries = self.queries(query)  # (N, query_len, heads, heads_dim)
        
        # einsum does matrix multiplication for query*keys for each training example with every other training example       
        energy = torch.einsum("nqhd, nkhd -> nhqk", [queries, keys])
        
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)
        
        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))
        
        # Normalize energy values
        attention = torch.softmax(energy// (self.embed_size ** (0.5)), dim = 3)
        # attention shape: (N, heads, query_len, key_len)
        
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim)
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # output after matrix multiply: (N, query_len, heads, head_dim)
        
        out = self.fc_out(out)
        return out

In [62]:
if __name__=='__main__':
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    
    x = torch.tensor([[1,5,9,4,3,2,5,10,3], [9,0,4,8,4,5,1,13,3]]).to(device)
    trg = torch.tensor([[3,0,5,7,1,0,6,11], [9,5,0,3,1,14,5,8]]).to(device)
    
    src_vocab_size = 20
    trg_vocab_size = 20
    src_pad_idx = 0
    trg_pad_idx = 0
    
    
    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device = device).to(device)
    
    out = model(x, trg[:, :-1])
    print(out.shape)

cpu
torch.Size([2, 7, 20])


<ipython-input-57-b6316df16e59>:43: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  attention = torch.softmax(energy// (self.embed_size ** (0.5)), dim = 3)
